In [5]:
import os
import time
import copy
import math
import pickle
import numpy as np
import numba
import pandas as pd
import pocomc as pc
from scipy.stats import qmc
import matplotlib.pyplot as plt
from multiprocessing import Pool
from numba import jit

N_PARTICLES = 1000
SEED=1
np.random.seed(seed=SEED)
n_cpus = os.cpu_count() 

In [6]:
def runge_kutta4_step(dydt, tn, yn, h, params):
    k1 = h * dydt(tn, yn, params)
    k2 = h * dydt(tn + 0.5 * h, yn + 0.5 * k1, params)
    k3 = h * dydt(tn + 0.5 * h, yn + 0.5 * k2, params)
    k4 = h * dydt(tn + h, yn + k3, params)
    return yn + (k1 + 2 * k2 + 2 * k3 + k4) / 6

def runge_kutta5_step(dydt, tn, yn, h, params):
    k1 = h * dydt(tn, yn, params)
    k2 = h * dydt(tn + 0.25 * h, yn + 0.25 * k1, params)
    k3 = h * dydt(tn + 3 * h / 8, yn + 3 * k1 / 32 + 9 * k2 / 32, params)
    k4 = h * dydt(tn + 12 * h / 13, yn + 1932 * k1 / 2197 - 7200 * k2 / 2197 + 7296 * k3 / 2197, params)
    k5 = h * dydt(tn + h, yn + 439 * k1 / 216 - 8 * k2 + 3680 * k3 / 513 - 845 * k4 / 4104, params)
    k6 = h * dydt(tn + 0.5 * h, yn - 8 * k1 / 27 + 2 * k2 - 3544 * k3 / 2565 + 1859 * k4 / 4104 - 11 * k5 / 40, params)
    return yn + 25 * (k1 / 216 + 125 * k3 / 216 + 81 * k4 / 216 - k5 / 216)

def rk45(dydt, y0, t0, duration, parameters, time_points):
    """
    Performs Runge-Kutta 4/5 integration and evaluates the solution at specific time points.
    (Works with Numba JIT for improved performance)

    Parameters:
        dydt (function): The differential equation function dy/dt = f(t, y, parameters).
        y0 (float or list): Initial value(s) of the dependent variable(s) y.
        t0 (float): Initial value of the independent variable t.
        duration (float): Total duration for integration.
        parameters (tuple): Parameters to pass to the differential equations function.
        time_points (list): Time points at which to evaluate the solution.

    Returns:
        Tuple: Two arrays containing the time values and corresponding y values.
    """
    time_points = np.append(time_points, t0)
    y_points = [y0]

    t = t0
    y = y0
    h = (time_points[1] - t0) if len(time_points) > 1 else 0.01
    params = parameters

    for i in range(len(time_points) - 1):
        while t < time_points[i + 1]:
            # Calculate the y value using Runge-Kutta 4
            y_rk4 = runge_kutta4_step(dydt, t, y, h, params)

            # Calculate the y value using Runge-Kutta 5
            y_rk5 = runge_kutta5_step(dydt, t, y, h, params)

            # Estimate the error and update the step size
            error = abs(y_rk5 - y_rk4)
            h_new = h * (1e-15 / max(1e-15, *error))**0.2
            h = min(h_new, time_points[i + 1] - t)  # Ensure step size doesn't exceed the next time point

            # Update time and y value
            t += h
            y = y_rk5

        y_points.append(y)

    return time_points, y_points


In [7]:
# Defining Model
class Model:
    def __init__(self, opts): #initial settings
        for key in opts: #loops for all labels in the list 'key'
            setattr(self, key, opts[key]) #creates a dictionary where 'key' are the list of labels & 'ops[key]' are the values

    def __call__(self, theta_new):
        theta_new = theta_new
        res = self.log_likelihood(theta_new)
        return res
    
    def run_sim(self, model_param = None, x0 = None): #takes in canidate parameters then solves the ode
        if model_param is None:
            model_param= self.theta_true[:self.ODE_params_n]  #sets the model_params to just the model parameters
        if x0 is None:
            x0 = self.x0 #if x0 not defined, default x0 to the model x0
        t0 = self.ts[0]
        tfin = self.ts[-1] #define the time span
        x_n = self.x_n 
        
        result_time, result_y = rk45(self.sys_fun, x0, t0, tfin, model_param, self.ts) 
            
        return result_time, result_y
        
    def log_prior(self, theta_new): 
        bools = [(low <= i <= high) for i,low,high in zip(theta_new, self.lower_bnds, self.upper_bnds)] #if generated values are within bounds
        all_in_range = np.all(bools) #if all values are true, then output is true
        if all_in_range: #if true
            return 0.0 #give 0
        return -np.inf #if even one parameter out of bounds, it's false, and returns -infinity

    def log_likelihood(self, theta_new): #how good is this canidate parameter fitting my data (maximize it)
        model_param = theta_new[:self.ODE_params_n] 
        if self.fit_x0: 
            x0 = theta_new[self.ODE_params_n:(self.ODE_params_n + self.x_n)] #sets x0 to 'theta_true' x0 values
        else:
            x0 = self.x0

        if self.fit_sigma:
            sigma = theta_new[-len(self.observable_index):] #observable index related to sigma
        else:
            sigma = [1] * len(self.observable_index) #makes all sigmas default to 1

        ts, y = self.run_sim(model_param=model_param, x0= x0) #sets y to the y results of solving ODE
        data = self.data #sets data
        if self.x_n > 1:
            y = np.transpose(y)

        # Calculate posterior; how good is parameter in terms of fitting the data
        term1 = -0.5 * np.log(2*np.pi*np.square(sigma))
        term2 = np.square(np.subtract(y, data)) / (2*np.square(sigma))
        logLH = np.sum(term1 - term2)
        return logLH

In [9]:
# Creating Model 2 Objects
# Model 2 Options
mod2_opts = {} #creates a dictionary
mod2_opts['theta_n'] = 8 #total number of values in big array
mod2_opts['ODE_params_n'] = 4 # num ODE params
mod2_opts['x_n'] = 2 # num species
mod2_opts['sigma_n'] = 2 # num sigmas
mod2_opts['theta_names'] = ['$k_1$', '$k_2$', '$k_3$', '$k_4$', '$x_1$', '$x_2$','$\sigma_1$','$\sigma_2$'] #names of the parameters (for dictionary)
mod2_opts['fit_x0'] = True # fit initial conditions?
mod2_opts['x0'] = [2, 0.25] # initial conditions (if given)
mod2_opts['fit_sigma'] = True #fit sigma?
mod2_opts['observable_index'] = [0, 1]
mod2_opts['theta_true'] = [8, 1, 1, 1, 2, 0.25, 0.3, 0.3] # true param values
mod2_opts['lower_bnds'] = [2, 0, 0, 0, -3, -3, 1E-3, 1E-3] #lower bounds
mod2_opts['upper_bnds'] = [20, 5, 5, 5, 3, 3, 1, 1] #upper bounds

# load data for model 2
mod2_df = pd.read_csv('m2_data.csv', header=0, delimiter=",") #reads in data file
mod2_opts['ts'] = mod2_df['t'].values #sets data under 't' in csv as 'ts'
mod2_opts['data'] = mod2_df[['x1', 'x2']].values #makes data array and reshapes it

def mod2_sys(t, y, theta): #makes a new function
    temp = np.empty(shape=(2), dtype=np.float64)
    x1, x2 = y 
    k1, k2, k3, k4 = theta #, x1, x2, sigma1, sigma2
    dxdt = tuple([(2 * k1 * x2) - (k2 * (x1**2)) - (k3 * x1 * x2) - (k4 * x1), (k2 * (x1**2)) - (k1 * x2)])
    return dxdt

mod2_opts['sys_fun'] = mod2_sys
mod2 = Model(mod2_opts)

result = mod2.run_sim([0,0,0,0.25], [15,.5]) #run the simulation on model 2 parameters

fig = plt.figure(figsize=(14,5), dpi=300)
fig.add_subplot(1, 2, 1)
plt.plot(result[0], result[1][:, 0], 'r-')
#plt.plot(mod2.ts, mod2.data[:, 0], 'ro')

fig.add_subplot(1, 2, 2)
plt.plot(result[0], result[1][:, 1], 'b-')
#plt.plot(mod2.ts, mod2.data[:, 1], 'bo');


TypeError: can't multiply sequence by non-int of type 'float'